<a href="https://colab.research.google.com/github/Ramshiv2798/python/blob/main/Credit_Card_Fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
organizations_mlg_ulb_creditcardfraud_path = kagglehub.dataset_download('organizations/mlg-ulb/creditcardfraud')

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data['Class'].value_counts()

In [ ]:
data.hist(bins = 30, figsize = (30, 30))

In [ ]:
data.describe()

In [ ]:
from sklearn.preprocessing import RobustScaler


new_data = data.copy()


scaler = RobustScaler()


amount_reshaped = new_data['Amount'].to_numpy().reshape(-1, 1)


new_data['Amount'] = scaler.fit_transform(amount_reshaped)


new_data['Amount'].hist()

In [ ]:
new_data['Amount'].describe()

In [ ]:
time = new_data['Time']
new_data['Time'] = (time - time.min()) / (time.max() - time.min())
new_data['Time']

In [ ]:
np.random.seed(42)
new_data_shuffle = new_data.sample(frac  = 1).reset_index(drop = True)
new_data_shuffle

In [ ]:
train, test, val = new_data_shuffle[:240000], new_data_shuffle[240000: 262000], new_data_shuffle[262000:]

train['Class'].value_counts(), test['Class'].value_counts(), val['Class'].value_counts()

In [ ]:
import numpy as np




train_shape, test_shape, val_shape = train.shape, test.shape, val.shape


print("Train shape:", train_shape)
print("Test shape:", test_shape)
print("Validation shape:", val_shape)

In [ ]:
x_train, y_train = train.iloc[:, :-1], train.iloc[:, -1]
x_test, y_test = test.iloc[:, :-1], test.iloc[:, -1]
x_val, y_val = val.iloc[:, :-1], val.iloc[:, -1]

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()

logistic.fit(x_train, y_train)

logistic.score(x_train, y_train)

In [ ]:
from sklearn.metrics import classification_report


target_labels = ['Not Fraud', 'Fraud']


report = classification_report(y_val, logistic.predict(x_val), target_names=target_labels)


print(report)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint


shallow_nn = Sequential()



shallow_nn.add(Input(shape=(x_train.shape[1],)))

shallow_nn.add(BatchNormalization())
shallow_nn.add(Dense(1, activation='sigmoid'))



checkpoint_callback = ModelCheckpoint(filepath='best_model.keras', monitor='val_accuracy', mode='max', save_best_only=True)


shallow_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


shallow_nn.summary()

In [ ]:
shallow_nn.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5, callbacks=[checkpoint_callback])

In [ ]:
(shallow_nn.predict(x_test).flatten() > 0.5).astype(int)

In [ ]:
y_pred_prob = shallow_nn.predict(x_val)


y_pred = (y_pred_prob > 0.5).astype(int)


report = classification_report(y_val, y_pred)


print(report)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


rf = RandomForestClassifier(max_depth=2, n_jobs=-1)
rf.fit(x_train, y_train)


y_pred = rf.predict(x_val)


report = classification_report(y_val, y_pred)


print(report)

In [ ]:
not_frauds = new_data.query('Class == 0')
frauds = new_data.query('Class == 1')

not_frauds['Class'].value_counts(), frauds['Class'].value_counts()

In [ ]:
balance_data = pd.concat([frauds, not_frauds.sample(len(frauds), random_state = 42)])

balance_data['Class'].value_counts()

In [ ]:
balance_data = balance_data.sample(frac = 1, random_state = 42)
balance_data

In [ ]:
from sklearn.model_selection import train_test_split


balance_data_numpy = balance_data.to_numpy()


X = balance_data_numpy[:, :-1]
y = balance_data_numpy[:, -1]


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)


X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


In [ ]:
pd.Series(y_train).value_counts()

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()

logistic.fit(X_train, y_train)

logistic.score(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report


target_labels = ['Not Fraud', 'Fraud']


report = classification_report(y_val, logistic.predict(X_val), target_names=target_labels)


print(report)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint


shallow_nn = Sequential()


shallow_nn.add(Input(shape=(X_train.shape[1],)))
shallow_nn.add(BatchNormalization())
shallow_nn.add(Dense(1, activation='sigmoid')) # Output layer with sigmoid activation for binary classification


checkpoint_callback = ModelCheckpoint(filepath='best_model.keras', monitor='val_accuracy', mode='max', save_best_only=True)


shallow_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


shallow_nn.summary()

In [ ]:
shallow_nn.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, callbacks=[checkpoint_callback])

In [ ]:
y_pred_prob = shallow_nn.predict(X_val)


y_pred = (y_pred_prob > 0.5).astype(int)


report = classification_report(y_val, y_pred)


print(report)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


rf = RandomForestClassifier(max_depth=2, n_jobs=-1)
rf.fit(X_train, y_train)


y_pred = rf.predict(X_val)


report = classification_report(y_val, y_pred)


print(report)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report


gbc = GradientBoostingClassifier(learning_rate=1, n_estimators=50)


gbc.fit(X_train, y_train)


y_pred = gbc.predict(X_val)

report = classification_report(y_val, y_pred)


print(report)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


svc = LinearSVC(class_weight='balanced')


svc.fit(X_train, y_train)


y_pred = svc.predict(X_val)


report = classification_report(y_val, y_pred)


print(report)